In [1]:
pip install -U openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [6]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [7]:
!wget https://www.nichibun-g.co.jp/textbooks/joho/2013_shidousho/download/text_25SJ_05.txt

--2024-01-26 11:59:56--  https://www.nichibun-g.co.jp/textbooks/joho/2013_shidousho/download/text_25SJ_05.txt
Resolving www.nichibun-g.co.jp (www.nichibun-g.co.jp)... 153.127.63.216
Connecting to www.nichibun-g.co.jp (www.nichibun-g.co.jp)|153.127.63.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5059 (4.9K) [text/plain]
Saving to: ‘text_25SJ_05.txt’

text_25SJ_05.txt    100%[===================>]   4.94K  --.-KB/s    in 0s      

2024-01-26 11:59:57 (280 MB/s) - ‘text_25SJ_05.txt’ saved [5059/5059]



In [51]:
import numpy as np

texts = []
with open('text_25SJ_05.txt', encoding='utf8') as f:
  for r in f:
    texts.append(r.replace("\u3000","").replace("\ufeff", ""))


In [ ]:
texts

In [75]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [76]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(''.join(texts), "cl100k_base")

1770

In [77]:
0.00002 * 148 * 1.77

0.005239200000000001

In [78]:
len(texts)

50

In [79]:
# API利用してembedding取得
embeddings = [get_embedding(t) for t in texts]

In [80]:
import numpy as np

embed_np = np.array(embeddings)

In [86]:
ret_only = []
for i, t in enumerate(texts):
  if t == "\n":
    ret_only.append(i)
    print("改行のみ")

改行のみ
改行のみ
改行のみ
改行のみ
改行のみ
改行のみ
改行のみ
改行のみ


In [82]:
embed_np.shape

(50, 1536)

In [61]:
from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
user_input = "予約システムの故障で，多くの人は何ができなくなった？"

In [91]:
# 入力の埋め込み取得、numpy配列へ変換
input_vector = np.array(get_embedding(user_input))
in_vec_np = np.array(input_vector)

In [92]:
user_input

'予約システムの故障で，多くの人は何ができなくなった？'

In [90]:
np.array(input_vector).shape

(1536,)

In [63]:
# 入力クエリと距離の近いトップ3レコードを取得してcontextへ格納
dot_products = embed_np@in_vec_np
topn = 3
cnt = 0
context = ""
for i, idx in enumerate(np.argsort(embed_np@in_vec_np)[::-1]):
  if idx in ret_only:
    continue
  if cnt == 3:
    break
  #print(i, dot_products[idx], texts[idx])
  context+=texts[idx]
  cnt += 1

In [69]:
user_input

'予約システムの故障で，多くの人は何ができなくなった？'

In [64]:
context

'予約システムの故障で，多くの人が鉄道や飛行機に乗れなくなった事例はいくつもある。銀行のオンラインシステムの障害で，入金・送金・支払いができなくなった事例も少なくない。\n・通信システムのトラブル\n道具が強力であればあるほど，故障したり悪用されたりしたときの問題は大きくなる。自然災害やプログラムのバグ，人間の操作ミスなどで情報システムが止まってしまうことがある。たとえば交通機関に関係する情報システムが停止すると，安全な運行ができなくなり，何万もの人々が影響を受ける。情報システムを破壊するような犯罪が起こるとその被害は甚大だ。\n'

In [95]:
user_prompt = f"""(#ユーザからの問い)に次の(#コンテキスト)に従って回答せよ。
# ユーザからの問い
{user_input}
# コンテキスト
{context}

"""
print(user_prompt)

(#ユーザからの問い)に次の(#コンテキスト)に従って回答せよ。
# ユーザからの問い
予約システムの故障で，多くの人は何ができなくなった？
# コンテキスト
予約システムの故障で，多くの人が鉄道や飛行機に乗れなくなった事例はいくつもある。銀行のオンラインシステムの障害で，入金・送金・支払いができなくなった事例も少なくない。
・通信システムのトラブル
道具が強力であればあるほど，故障したり悪用されたりしたときの問題は大きくなる。自然災害やプログラムのバグ，人間の操作ミスなどで情報システムが止まってしまうことがある。たとえば交通機関に関係する情報システムが停止すると，安全な運行ができなくなり，何万もの人々が影響を受ける。情報システムを破壊するような犯罪が起こるとその被害は甚大だ。





In [93]:
# RAGを使わない回答
completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "あなたは役に立つアシスタントだ。"},
    {"role": "user", "content": user_input}
  ]
)

In [94]:
print(completion.choices[0].message.content)

予約システムの故障によって多くの人ができなくなることは、以下のようなことが考えられます：

1. ホテルや旅行の予約：予約システムが機能しないと、ホテルの部屋や旅行プランの予約が難しくなります。これにより旅行の計画が大きく影響を受ける可能性があります。

2. レストランの予約：特に人気のレストランでは、予約無しでは入店が難しい場合があります。システムの故障により、希望する日時にレストランを予約することができなくなる可能性があります。

3. 医療機関での診療予約：医療機関でも予約システムが広く使われており、システムのダウンは診療予約の取得を困難にします。これにより、患者は必要な診療を受けられないか、診療を受けるための待ち時間が長くなる可能性があります。

4. イベントや公演のチケット予約：コンサート、映画、スポーツイベントなどの公演やイベントのチケット予約も、オンライン予約システムに依存しています。故障により、チケットの購入や予約ができなくなり、観客がイベントを楽しむ機会を逃すことになるかもしれません。

5. 交通機関の予約：飛行機、列車、バスなどの交通機関の座席を予約する際にも、予約システムが使われます。システムの故障は、旅行者が移動手段を確保するのを難しくするだけでなく、既に予約した座席の管理や変更もできなくなる可能性があります。

いずれの場合も、予約システムの故障は不便をもたらし、計画や日常生活に支障をきたす可能性があります。


In [71]:
# RAGを使った回答
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "あなたは役に立つアシスタントだ。与えられた(#コンテキスト)に従って回答ができる"},
    {"role": "user", "content": user_prompt}
  ]
)

ChatCompletionMessage(content='予約システムの故障により、多くの人は鉄道や飛行機に乗ることができなくなったり、銀行のオンラインシステムの障害で入金・送金・支払いができなくなったりしました。予約システムやオンラインシステムは、通信システムのトラブルや自然災害、プログラムのバグ、人間の操作ミスなどにより停止することがあります。その影響は大きく、多くの人々が困ることとなります。また、情報システムを悪用する犯罪行為が行われると、さらなる被害が生じる可能性があります。', role='assistant', function_call=None, tool_calls=None)


In [72]:
print(completion.choices[0].message.content)

予約システムの故障により、多くの人は鉄道や飛行機に乗ることができなくなったり、銀行のオンラインシステムの障害で入金・送金・支払いができなくなったりしました。予約システムやオンラインシステムは、通信システムのトラブルや自然災害、プログラムのバグ、人間の操作ミスなどにより停止することがあります。その影響は大きく、多くの人々が困ることとなります。また、情報システムを悪用する犯罪行為が行われると、さらなる被害が生じる可能性があります。
